In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Steps
1. data preprocessing 
* importing the files(Dataset)
* handeling missing data
* splitting the dataset into training set and test set
* encoding categorical data
* Feature scaling
2. model training
3. model evaluation

# Importing dataset

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv", index_col='PassengerId')
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

# Dropping some coluoms of our tables

In [ ]:
z=train_data.drop(['Name','Ticket','Cabin'],axis=1)
t=test_data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [ ]:
z.head()

In [ ]:
t.head()

* x is the matrix of the features
* y is the independent variable
* iloc is used to locate index (Rows and Columns)

In [ ]:
x=z.iloc[:,1:]   
y=train_data.iloc[:,0].values

# Check if there is a missing data
there is 2 ways how to handle our missing data which is either by:
* removing it
* replacing it by -Average or - median or - most used,the most classic way to handle this is to get average

In [ ]:
x.isnull().sum()

In [ ]:
#checking 
t.isnull().sum()

# Handeling the missing data found

In [ ]:
from sklearn.impute import SimpleImputer
theimputer = SimpleImputer(missing_values=np.nan,strategy='median')
theimputer.fit(x.iloc[:,2:3])
x.iloc[:,2:3]=theimputer.transform(x.iloc[:,2:3])
#filling the missing data of Embarked coloum
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(x.iloc[:,[6]])
x.iloc[:,[6]]=imp.transform(x.iloc[:,[6]])

In [ ]:
#checking 
x.isnull().sum()

filling the missing data of Age and fare coloum

In [ ]:
#filling the missing data of Age coloum
theimputer.fit(t.iloc[:,[2]])
t.iloc[:,[2]]=theimputer.transform(t.iloc[:,[2]])
#filling the missing data of Fare coloum using mean strategy
theimputern2 = SimpleImputer(missing_values=np.nan,strategy='mean')
theimputern2.fit(t.iloc[:,[5]])
t.iloc[:,[5]]=theimputern2.transform(t.iloc[:,[5]])

In [ ]:
t.isnull().sum()

In [ ]:
x.head()

In [ ]:
t.head()

# Splitting the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=1)

# Encoding Categorical data
In Machine learning we are require that input and output variables are numbers
* Here we have the sex Column containg letters soit must be encoded to numbers before we can use it to fit and evaluate our mode

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
column=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Sex','Embarked'])],remainder='passthrough')
xtrain= np.array(column.fit_transform(xtrain))
xtest=np.array(column.fit_transform(xtest))
#Encoding Categorical data
t= np.array(column.fit_transform(t))

In [ ]:
print(xtrain)

In [ ]:
print(xtest)

In [ ]:
print(t)

# Feature scaling 

In [ ]:
#Feature scaling is essential as the range of all features should be in fixed range
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#features to be scaled : age and fare
xtrain= sc.fit_transform(xtrain)
xtest = sc.transform(xtest)
#Scaling the data of test
t = sc.transform(t)

# Random Forest Classifier

In [ ]:
#For my model Random Forest Classifier
#To produce a more accurate result, it builds numerous decision trees and then merges them together.
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 1)
classifier.fit(xtrain, ytrain)

In [ ]:
ypredict = classifier.predict(xtest)
ypredict


# Confusion Matrix

In [ ]:
#Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
confussionMatrix = confusion_matrix(ytest, ypredict)
print(confussionMatrix)
accuracy_score(ytest, ypredict)

In [ ]:
predict = classifier.predict(t)
predict

# submission

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predict})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")